In [173]:
from random import random
from math import ceil
from functools import reduce
from collections import namedtuple, deque
from queue import PriorityQueue

import numpy as np
from tqdm.auto import tqdm

In [174]:
State = namedtuple('State', ['taken', 'not_taken'])

def covered(state):
    return reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )


def goal_check(state):
    return np.all(covered(state))

In [175]:
PROBLEM_SIZE = 6
NUM_SETS = 10
SETS = tuple(np.array([random() < 0.3 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
assert goal_check(State(set(range(NUM_SETS)), set())), "Probelm not solvable"
SETS

(array([False, False,  True, False, False, False]),
 array([ True,  True, False, False, False, False]),
 array([False, False, False, False, False, False]),
 array([ True, False, False, False, False, False]),
 array([False,  True, False, False,  True, False]),
 array([False, False, False,  True, False, False]),
 array([False,  True, False, False, False, False]),
 array([ True,  True, False,  True, False,  True]),
 array([False,  True, False, False, False, False]),
 array([False, False, False, False,  True, False]))

In [184]:
def h(state):
    largest_set_size = max(sum(s) for s in SETS)
    missing_size = PROBLEM_SIZE - sum(covered(state))
    optimistic_estimate = ceil(missing_size / largest_set_size)
    return optimistic_estimate


def h2(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    largest_set_size = max(sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS)
    missing_size = PROBLEM_SIZE - sum(already_covered)
    optimistic_estimate = ceil(missing_size / largest_set_size)
    return optimistic_estimate


def h3(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(already_covered)
    candidates = sorted((sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS), reverse=True)
    taken = 1
    while sum(candidates[:taken]) < missing_size:
        taken += 1
    return taken

def h4(state):  #This method works like Greedy algorithm, it is not the best like h3, but it is another approach
                #I need to point out that somethimes it takes more tiles than h3 but it often takes less steps than h3
    already_covered = covered(state)
    if np.all(already_covered):
        return 0

    to_cover = PROBLEM_SIZE - np.sum(already_covered)   #number of tiles to cover

    taken = 0

    while np.any(already_covered == False) and taken < to_cover:
        # search set not already covered
        max_covering_set = None
        max_covering_size = -1
        for i, s in enumerate(SETS):
            if i not in state.taken:
                covering_size = np.sum(np.logical_and(s, np.logical_not(already_covered)))
                if covering_size > max_covering_size:   #if it is not already covered
                    max_covering_size = covering_size
                    max_covering_set = s

        if max_covering_set is not None:    #if it is covered max_covering_set will be None
            state.taken.add(i)
            already_covered = np.logical_or(already_covered, max_covering_set)
            taken += 1
        else:
            break       #we don't have to take any other set 

    return taken



def f(state):
    return len(state.taken) + h4(state)

In [185]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f(state), state))

counter = 0
_, current_state = frontier.get()
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put((f(new_state), new_state))
        _, current_state = frontier.get()
        print(current_state)
        pbar.update(1)

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")
print(current_state)

8it [00:00, 962.05it/s]

State(taken={9}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8})
State(taken={0, 9}, not_taken={1, 2, 3, 4, 5, 6, 7, 8, 9})
State(taken={9, 7}, not_taken={0, 1, 2, 3, 4, 5, 6, 8})
State(taken={0, 9}, not_taken={1, 2, 3, 4, 5, 6, 7, 8})
State(taken={0, 9}, not_taken={1, 2, 3, 4, 5, 6, 7, 8})
State(taken={9, 7}, not_taken={0, 1, 2, 3, 4, 5, 6, 8, 9})
State(taken={9, 7}, not_taken={0, 1, 2, 3, 4, 5, 6, 8})
State(taken={0, 9, 7}, not_taken={1, 2, 3, 4, 5, 6, 8})
Solved in 8 steps (3 tiles)
State(taken={0, 9, 7}, not_taken={1, 2, 3, 4, 5, 6, 8})
